In [ ]:
import pandas as pd

In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
    lines = list(map(lambda x: [*x.strip()], lines))
board = pd.DataFrame(lines)
board

In [ ]:
from collections import namedtuple


Position = namedtuple("Position", ["column", "row"])
# startingPosition = Position(1,1)
startingPosition = (Position(column=79, row=64))
startingPositionSymbol = "|"

In [ ]:
def getCurrentSymbol(currentPosition):
    currentSymbol = board.iloc[currentPosition.row, currentPosition.column]
    currentSymbol = startingPositionSymbol if currentPosition == startingPosition else currentSymbol
    return currentSymbol

In [ ]:
#Task 2
def getLeftPoint(point):
    return Position(point.column-1, point.row)
def getRightPoint(point):
    return Position(point.column+1, point.row)
def getTopPoint(point):
    return Position(point.column, point.row-1)
def getBottomPoint(point):
    return Position(point.column, point.row+1)
def getTopLeftPoint(point):
    return Position(point.column-1, point.row-1)
def getTopRightPoint(point):
    return Position(point.column+1, point.row-1)
def getBottomRightPoint(point):
    return Position(point.column+1, point.row+1)
def getBottomLeftPoint(point):
    return Position(point.column-1, point.row+1)

def getNeighbors(point):
    startingPoints = [
        getLeftPoint(point),
        getTopLeftPoint(point),
        getTopPoint(point),
        getTopRightPoint(point),
        getRightPoint(point),
        getBottomRightPoint(point),
        getBottomPoint(point),
        getBottomLeftPoint(point),
    ]
    return startingPoints

def getOutsidePointsOfCorner(point):
    cornerSymbol = getCurrentSymbol(point)
    a = {
        "L": [getBottomPoint(point), getBottomLeftPoint(point), getLeftPoint(point)],
        "J": [getRightPoint(point), getBottomRightPoint(point), getBottomPoint(point)], 
        "7": [getTopPoint(point), getTopRightPoint(point), getRightPoint(point)], 
        "F": [getLeftPoint(point), getTopLeftPoint(point), getTopPoint(point)], 
    }
    return a[cornerSymbol]

def getInsidePointsOfCorner(point):
    cornerSymbol = getCurrentSymbol(point)
    a = {
        "L": [getTopRightPoint(point)],
        "J": [getTopLeftPoint(point)], 
        "7": [getBottomLeftPoint(point)], 
        "F": [getBottomRightPoint(point)], 
    }
    return a[cornerSymbol]

def getEntryDirectionOfStartingPoint(point):
    symbol = getCurrentSymbol(point)
    directions = {
        "|": "D",
        "-": "L",
        "L": "L",
        "J": "D",
        "7": "R",
        "F": "U",
    }
    return directions[symbol]

def getNextPoint(point, direction):
    if direction == "U": return getTopPoint(point) 
    if direction == "D": return getBottomPoint(point)
    if direction == "L": return getLeftPoint(point)
    if direction == "R": return getRightPoint(point)

def getExitDirection(point, entryDirection):
    symbol = getCurrentSymbol(point)
    directionsPerSymbol = {
        "|": {
            "U": "U",
            "D": "D",
        },
        "-": {
            "L": "L",
            "R": "R",
        },
        "L": {
            "L": "U",
            "D": "R",
        },
        "J": {
            "D": "L",
            "R": "U",
        },
        "7": {
            "R": "D",
            "U": "L",
        },
        "F": {
            "L": "D",
            "U": "R",
        },
    }
    return directionsPerSymbol[symbol][entryDirection]

def getParallelPathsForCurrentPoint(point, direction):
    #returns a list of two paths
    #the first list are the points that can be added to the parallel path that is on the left-hand side
    #the second list are the points that can be added to the parallel path that is ln the right-hand side
    symbol = getCurrentSymbol(point)
   
    if symbol == "|": 
        directionMap = {
            "U": [[getLeftPoint(point)], [getRightPoint(point)]],
            "D": [[getRightPoint(point)], [getLeftPoint(point)]],
        }
        return directionMap[direction]
    elif symbol == "-": 
        directionMap = {
            "L": [[getBottomPoint(point)], [getTopPoint(point)]],
            "R": [[getTopPoint(point)], [getBottomPoint(point)]],
        }
        return directionMap[direction]
    elif symbol == "L": 
        directionMap = {
            "L": [getOutsidePointsOfCorner(point), getInsidePointsOfCorner(point)],
            "D": [getInsidePointsOfCorner(point), getOutsidePointsOfCorner(point)],
        }
        return directionMap[direction]
    elif symbol == "J":
        directionMap = {
            "D": [getOutsidePointsOfCorner(point), getInsidePointsOfCorner(point)],
            "R": [getInsidePointsOfCorner(point), getOutsidePointsOfCorner(point)],
        }
        return directionMap[direction]
    elif symbol == "7": 
        directionMap = {
            "R": [getOutsidePointsOfCorner(point), getInsidePointsOfCorner(point)],
            "U": [getInsidePointsOfCorner(point), getOutsidePointsOfCorner(point)],
        }
        return directionMap[direction]
    elif symbol == "F": 
        directionMap = {
            "U": [getOutsidePointsOfCorner(point), getInsidePointsOfCorner(point)],
            "L": [getInsidePointsOfCorner(point), getOutsidePointsOfCorner(point)],
        }
        return directionMap[direction]
    
def walkAlongPath(startingPoint):
    direction = getEntryDirectionOfStartingPoint(startingPoint)
    leftParallelPath = []
    rightParallelPath = []
    path = []
    
    currentPoint = startingPoint
    
    while currentPoint != startingPoint or not leftParallelPath:
        # get parallel points and add them to their path
        path.append(currentPoint)
        leftPoints, rightPoints = getParallelPathsForCurrentPoint(currentPoint, direction)
        leftParallelPath += leftPoints
        rightParallelPath += rightPoints
        
        exitDirectionOfCurrentPoint = getExitDirection(currentPoint, direction)
        nextPoint = getNextPoint(currentPoint, exitDirectionOfCurrentPoint)
        
        direction = exitDirectionOfCurrentPoint
        currentPoint = nextPoint
    return path, leftParallelPath, rightParallelPath

def initLookupBoard(board, path):
    lookupBoard = board.copy()
    for col in lookupBoard.columns:
        lookupBoard[col].values[:] = "."
    
    for point in path:
        lookupBoard.iloc[point.row, point.column] = "P"
    return lookupBoard

def searchFromStartingPoint(point, lookupBoard):
    searchTouchedOutside=False
    nextPoints = [point]
    
    while nextPoints:
        currentPoint = nextPoints.pop()
        if currentPoint.row<0 or currentPoint.column<0 or currentPoint.row >= board.shape[0] or currentPoint.column >= board.shape[1]:
            searchTouchedOutside = True
        elif lookupBoard.iloc[currentPoint.row, currentPoint.column] == ".":
            lookupBoard.iloc[currentPoint.row, currentPoint.column] = "S"
            
            neighbors = getNeighbors(currentPoint)
            for neighbor in neighbors:
                if neighbor.row<0 or neighbor.column<0 or neighbor.row >= board.shape[0] or neighbor.column >= board.shape[1]:
                    searchTouchedOutside = True
                elif lookupBoard.iloc[neighbor.row, neighbor.column] == ".":
                    nextPoints.append(neighbor)
    
    mapperDict = {"S": "O"} if searchTouchedOutside else {"S": "I"}
    return lookupBoard.map(lambda value: mapperDict[value] if value in mapperDict else value)

path, leftPath, rightPath = walkAlongPath(startingPosition)

In [ ]:
#Task 1
from math import floor
cycleLength = len(path)
farthestPoint = floor(cycleLength/2)
farthestPoint

In [ ]:
#Task 2
finalBoard =pd.DataFrame()
leftPath = set(leftPath)-set(path)
rightPath = set(rightPath)-set(path)

for parallelPath in [leftPath, rightPath]:
    lookupBoard = initLookupBoard(board, path)
    for point in parallelPath:
        if "O" not in lookupBoard.values:
            lookupBoard = searchFromStartingPoint(point, lookupBoard)
    if "O" not in lookupBoard.values:
        finalBoard=lookupBoard
        break
        
finalBoard = finalBoard[finalBoard == "I"].map(lambda x: 1 if x == "I" else 0)

finalBoard.values.sum()